# RAG pipelines applied to Unipa's website

## Dependencies
- `haystack-ai` is the preview of Haystack 2.0
- `sentence_transformers` is needed for embeddings
- `transformers` is needed to use open-source LLMs
- `accelerate` and `bitsandbytes` are required to use quantized versions of these models (with smaller memory footprint)

In [34]:
%%capture
! pip install wikipedia haystack-ai transformers accelerate bitsandbytes sentence_transformers

## Setup

In [35]:
from IPython.display import Image
from pprint import pprint
import torch
import rich
import random

In [36]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import pandas as pd
import numpy as np

filepath = "/content/drive/MyDrive/output240617110842.json"
df = pd.read_json(filepath)
df.columns = ["title", "addr", "text"]

df

,title,addr,text
0,Università degli Studi di Palermo,https://www.unipa.it/,
1,Organi di Governo | Università degli Studi di ...,https://www.unipa.it/ateneo/OrganiDiGovernoECo...,
2,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
3,Presentazione | Università degli Studi di Palermo,https://www.unipa.it/ateneo/presentazione/,
4,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
...,...,...,...
6333,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Ca...
6334,| Università degli Studi di Palermo,https://www.unipa.it/amministrazione/rettorato...,
6335,Settore Comunicazione e URP | Settore Comunica...,https://www.unipa.it/amministrazione/rettorato...,
6336,Regolamenti per aree tematiche di interesse | ...,https://www.unipa.it/servizi/prevenzionedellac...,REGOLAMENTI PERSONALE DOCENTE E RICERCATORE


## Preprocessing

In [38]:
import re


def remove_dates(s: str) -> str:
    regex = r"\d{1,2}-(gen|feb|mar|apr|mag|giu|lug|ago|set|ott|nov|dic)-\d{4}"

    return re.sub(regex, "", s)


def remove_empty_docs(df):
    df["text"] = df["text"].replace("", np.nan)
    df.dropna(subset=["text"], inplace=True)

    return df


def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    df["text"] = df["text"].apply(lambda x: x.strip())
    df = remove_empty_docs(df)
    df["text"] = df["text"].apply(remove_dates)

    # specify minimum document length
    # most documents below threshold tend to be garbage documents
    df = df[df["text"].str.len() >= 200]

    df.reset_index(inplace=True, drop=True)

    return df

In [39]:
df = preprocess(df)
df

,title,addr,text
0,Fatturazione elettronica | Università degli St...,https://www.unipa.it/target/imprese/informazio...,Il D.M 55 del 3 aprile 2013 prevede l'obbligo ...
1,Credits | Università degli Studi di Palermo,https://www.unipa.it/credits.html,I contenuti della home page e delle relative s...
2,Sostegno allo studio | Centro per l’innovazion...,https://www.unipa.it/strutture/cimdu/Sostegno-...,"Da 15 anni ItaStra, Scuola di Lingua Italian..."
3,PNRR | PNRR | Università degli Studi di Palermo,https://www.unipa.it/progetti/pnrr/,Il Piano Nazionale di Ripresa e Resilienza - P...
4,Corsi di preparazione alle prove di accesso A....,https://www.unipa.it/Corsi-di-preparazione-all...,Sono aperte le iscrizioni all’edizione inverna...
...,...,...,...
2774,U.O. Didattica e Internazionalizzazione - Desc...,https://www.unipa.it/strutture/cimdu/U.O.-Dida...,Il Responsabile dell’U.O. Didattica e Interna...
2775,Ricercatori neoassunti | Università degli Stud...,https://www.unipa.it/strutture/cimdu/ricercato...,Per restare sempre aggiornati è possiblie iscr...
2776,Declaratorie della U.O. | Centro per l’innovaz...,https://www.unipa.it/strutture/cimdu/Declarato...,La U.O. Didattica e Internazionalizzazione si...
2777,Calendari didattici DARCH | Centro per l’innov...,https://www.unipa.it/strutture/cimdu/Calendari...,Calendario didattico DARCH A.A. 2024/2025 Cal...


In [40]:
from haystack.dataclasses import Document

titles = list(df["title"].values)
texts = list(df["text"].values)
urls = list(df["text"].urls)
raw_docs = []
for title, text, url in zip(titles, texts, urls):
    raw_docs.append(Document(content=text, meta={"name": title or "", "url": url}))

## Indexing Pipeline

In [41]:
from haystack import Pipeline
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.preprocessors import DocumentCleaner, DocumentSplitter
from haystack.components.embedders import (
    SentenceTransformersTextEmbedder,
    SentenceTransformersDocumentEmbedder,
)
from haystack.components.writers import DocumentWriter
from haystack.document_stores.types import DuplicatePolicy
from haystack.utils import ComponentDevice

In [42]:
document_store = InMemoryDocumentStore(embedding_similarity_function="cosine")

Our indexing Pipeline transform the original Documents and save them in the Document Store.

It consists of several components:

- `DocumentCleaner`: performs a basic cleaning of the Documents
- `DocumentSplitter`: chunks each Document into smaller pieces (more appropriate for semantic search and RAG)
- `SentenceTransformersDocumentEmbedder`:
  - represent each Document as a vector (capturing its meaning).
  - we choose a good but not too big model from [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard).
  - Also the metadata `title` is embedded, because it contains relevant information (`metadata_fields_to_embed` parameter).
  - We use the GPU for this expensive operation (`device` parameter).
- `DocumentWriter` just saves the Documents in the Document Store

In [43]:
indexing = Pipeline()
indexing.add_component("cleaner", DocumentCleaner())
indexing.add_component(
    "splitter", DocumentSplitter(split_by="sentence", split_length=2)
)
indexing.add_component(
    "doc_embedder",
    SentenceTransformersDocumentEmbedder(
        model="thenlper/gte-large",
        device=ComponentDevice.from_str("cuda:0"),
        meta_fields_to_embed=["title"],
    ),
)
indexing.add_component(
    "writer",
    DocumentWriter(document_store=document_store, policy=DuplicatePolicy.OVERWRITE),
)

indexing.connect("cleaner", "splitter")
indexing.connect("splitter", "doc_embedder")
indexing.connect("doc_embedder", "writer")

🚅 Components
  - cleaner: DocumentCleaner
  - splitter: DocumentSplitter
  - doc_embedder: SentenceTransformersDocumentEmbedder
  - writer: DocumentWriter
🛤️ Connections
  - cleaner.documents -> splitter.documents (List[Document])
  - splitter.documents -> doc_embedder.documents (List[Document])
  - doc_embedder.documents -> writer.documents (List[Document])

In [44]:
indexing.run({"cleaner": {"documents": raw_docs}})

Batches:   0%|          | 0/948 [00:00<?, ?it/s]

{'writer': {'documents_written': 30317}}

Let's inspect the total number of chunked Documents and examine a Document

In [45]:
len(document_store.filter_documents())

25617

In [46]:
document_store.filter_documents()[0].meta

{'name': 'Fatturazione elettronica | Università degli Studi di Palermo',
 'source_id': 'fb89b71ef4cae3fde427ba015dc85455cdf30a511926b9ccee38c27ec93a7c34',
 'page_number': 1}

In [47]:
pprint(document_store.filter_documents()[0])
print(len(document_store.filter_documents()[0].embedding))  # embedding size

Document(id=bc2a342a7b442e56d4b09526918e347f4e7ccfc79dcd55f5fa700142fad4a87d, content: 'Il D.M 55 del 3 aprile 2013 prevede l'obbligo della fatturazione elettronica in tutti i rapporti con...', meta: {'name': 'Fatturazione elettronica | Università degli Studi di Palermo', 'source_id': 'fb89b71ef4cae3fde427ba015dc85455cdf30a511926b9ccee38c27ec93a7c34', 'page_number': 1}, embedding: vector of size 1024)
1024


## RAG Pipeline

### `HuggingFaceLocalGenerator` with `zephyr-7b-beta`

- To load and manage Open Source LLMs in Haystack 2.0, we can use the `HuggingFaceLocalGenerator`.

- The LLM we choose is [Zephyr 7B Beta](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), a fine-tuned version of Mistral 7B V.01 that focuses on helpfulness and outperforms many larger models on the MT-Bench and AlpacaEval benchmarks; the model was fine-tuned by the Hugging Face team.

- Since we are using a free Colab instance (with limited resources), we load the model using **4-bit quantization** (passing the appropriate `huggingface_pipeline_kwargs` to our Generator).
For an introduction to Quantization in Hugging Face Transformers, you can read [this simple blog post](https://huggingface.co/blog/merve/quantization).



In [48]:
from haystack.components.generators import HuggingFaceLocalGenerator

In [49]:
generator = HuggingFaceLocalGenerator(
    "HuggingFaceH4/zephyr-7b-beta",
    huggingface_pipeline_kwargs={
        "device_map": "auto",
        "model_kwargs": {
            "load_in_4bit": True,
            "bnb_4bit_use_double_quant": True,
            "bnb_4bit_quant_type": "nf4",
            "bnb_4bit_compute_dtype": torch.bfloat16,
        },
    },
    generation_kwargs={"max_new_tokens": 350},
)

In [50]:
generator.warm_up()

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

### `PromptBuilder`

 It's a component that renders a prompt from a template string using Jinja2 engine.

 Let's setup our prompt builder, with a format like the following (appropriate for Zephyr):

 `"<|system|>\nSYSTEM MESSAGE</s>\n<|user|>\nUSER MESSAGE</s>\n<|assistant|>\n"`

In [51]:
from haystack.components.builders import PromptBuilder

prompt_template = """<|system|>Using the information contained in the context, give a comprehensive answer to the question.
If the answer is contained in the context, also report the source URL.
If the answer cannot be deduced from the context, do not give an answer.</s>
<|user|>
Context:
  {% for doc in documents %}
  {{ doc.content }} URL:{{ doc.meta['url'] }}
  {% endfor %};
  Question: {{query}}
  </s>
<|assistant|>
"""
prompt_builder = PromptBuilder(template=prompt_template)

### Pipeline Creation

In [52]:
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever

Our RAG Pipeline finds Documents relevant to the user query and pass them to the LLM to generate a grounded answer.

It consists of several components:

- `SentenceTransformersTextEmbedder`: represent the query as a vector (capturing its meaning).
- `InMemoryEmbeddingRetriever`: finds the (top 5) Documents that are most similar to the query vector
- `PromptBuilder`
- `HuggingFaceLocalGenerator`

In [53]:
rag = Pipeline()
rag.add_component(
    "text_embedder",
    SentenceTransformersTextEmbedder(
        model="thenlper/gte-large", device=ComponentDevice.from_str("cuda:0")
    ),
)
rag.add_component(
    "retriever", InMemoryEmbeddingRetriever(document_store=document_store, top_k=5)
)
rag.add_component("prompt_builder", prompt_builder)
rag.add_component("llm", generator)

rag.connect("text_embedder", "retriever")
rag.connect("retriever.documents", "prompt_builder.documents")
rag.connect("prompt_builder.prompt", "llm.prompt")

🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: HuggingFaceLocalGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (List[float])
  - retriever.documents -> prompt_builder.documents (List[Document])
  - prompt_builder.prompt -> llm.prompt (str)

## Questions

In [54]:
def get_generative_answer(query):

    results = rag.run(
        {"text_embedder": {"text": query}, "prompt_builder": {"query": query}}
    )

    answer = results["llm"]["replies"][0]
    rich.print(answer)

In [55]:
get_generative_answer(
    "What are the courses of Data, Algorithm and Machine Intelligence at Unipa?"
)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the context provided, the courses offered under the program of Data, Algorithm and Machine Intelligence at
Unipa are:

1. Combinatorial and Probabilistic Algorithms
2. Big data management
3. Multisensory data exploration
4. Knowledge representation and reasoning
5. Cybersecurity
6. Information theory and data compression
7. Data encryption and codes
8. Cloud and high performance computing
9. Complex networks
10. Artificial intelligence and deep learning
11. Machine intelligence for optimization
12. Information retrieval and natural language processing
13. Pattern Discovery for Life Sciences

These courses are part of the study program LM 18, which requires one of the following qualifications: Italian 
Bachelor (Laurea) in L-31 scienze e tecnologie informatiche; L-35 scienze matematiche; L-8 ingegneria 
dell’informazione; L-41 statistica; L-30 scienze e tecnologie fisiche; Any Italian Bachelor (Laurea) or Master 
(Laurea Magistrale) including 12CFU (ECTS credits) of Mathematics and Physics, and 24 CFU of Computer and 
Information Sciences; Any foreign Degree that is equivalent to those described in points a) and b). Additionally, 
students are required to have an English language proficiency level of B2 according to the Common European 
Framework of Reference for Languages. The URL for more information about the study program LM 18 is: 
https://unipa.it/course/view.

In [56]:
get_generative_answer("Who were the deans of University of Palermo?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

The provided context lists a comprehensive list of rectors, or university presidents, of the University of Palermo 
for various historical periods. The list includes:

- Regno di Sicilia:
  - Gabriele Castelli (1781-1800)
  - Gregorio Speciale (1801-1805)
  - Raimondo Palermo (1806-1816)

- Regno delle Due Sicilie:
  - Raimondo Palermo (1816-1840)

- Regno d'Italia:
  - Alessandro Casano (1841-1843)
  - Giuseppe D'Agostino (1844-1855)
  - Giovanni Laviosa (1856-1858)
  - Giovanni Cumbo (1858-1860)
  - Filippo Casoria (1860-1861)
  - Salvatore Cacopardo (1861-1862)
  - Nicolò Musumeci (1863-1865)
  - Stanislao Cannizzaro (1866-1868)
  - Giuseppe Albeggiani (1869-1874)
  - Gaetano Giorgio Gemmellaro (1874-1876)
  - Antonino Garajo (1877-1

In [57]:
get_generative_answer("Tell me about Data, Algorithms and Machine Intelligence")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Based on the context provided, the Master's Degree program in Data, Algorithms, and Machine Intelligence offers a 
comprehensive curriculum that provides students with theoretical, experimental, and practical skills in data 
representation and computation for efficient and intelligent information processing. The program covers transversal
topics in data, algorithms, and machine intelligence, as well as in-depth studies in application areas where data 
are big, relations are complex, and machine-learning structures are deep. Specifically, classes taught in English 
include Combinatorial and Probabilistic Algorithms, Big Data Management, Multisensory Data Exploration, Knowledge 
Representation and Reasoning, Cybersecurity, Information Theory and Data Compression, Data Encryption and Codes, 
Cloud and High Performance Computing, Complex Networks, Artificial Intelligence and Deep Learning, Machine 
Intelligence for Optimization, Information Retrieval and Natural Language Processing, and Pattern Discovery for 
Life Sciences. To be eligible for the program, applicants must meet one of the following requirements: an Italian 
Bachelor (Laurea) in L-31 Informatics, L-35 Mathematics, L-8 Information Engineering, L-41 Statistics, L-30 
Physics, or any Italian Bachelor (Laurea) or Master (Laurea Magistrale) including 12 CFU (ECTS credits) of 
Mathematics and Physics, and 24 CFU of Computer and Information Sciences, or any foreign Degree that is equivalent 
to those described in points a) and b). Additionally, applicants must have a B2 level of English according to the 
Common European Framework of Reference for Languages. For more information,

In [59]:
get_generative_answer("Tell me about Unipa's history")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

I'm sorry but the provided context does not contain enough information to provide a comprehensive answer about 
Unipa's history. The context only provides URLs for Unipa, but without visiting those websites, it's impossible to 
determine the company's history. I would suggest visiting the Unipa website or doing further research to learn more
about their history. Here is the URL for their website: https://www.unipa.it/en/homepage/

Source URL: https://www.unipa.it/en/homepage/